In [ ]:
from swagger_parser import SwaggerParser

parser = SwaggerParser(swagger_path='swagger.json')  # Init with file

# Get an example of dict for the definition Foo
parser.definitions_example.get('Pet')

In [1]:
import json

with open("swagger.json","r") as infile:
    swagger = json.load(infile)

In [2]:
from faker import Faker
from chance import chance
import random
fake = Faker()



In [38]:
property_generators = {
    "name":lambda x: fake.name(),
    "date-time": lambda x: fake.date_time(),
    "username":lambda x: chance.email(),
    "email": lambda x: chance.email(),
    "id":lambda x: fake.uuid4(),
    "quantity":lambda x: random.randint(0,100),
    "integer":lambda x: random.randint(0,100),
    "bool": lambda x: chance.boolean(),
    "boolean": lambda x: chance.boolean(),
    "firstname": lambda x: fake.name().split(" ")[0],
    "lastname": lambda x: fake.name().split(" ")[-1],
    "password": lambda x: fake.password(),
    "phone": lambda x: fake.phone_number(),
    "url": lambda x: chance.url(),
    "enum": lambda x: random.choice(x[0]["enum"]),
    "array": lambda x: [x[1] + str(i) for i in range(random.randint(1,10))],
    "$ref": lambda x: list(generate(x[0]["$ref"].split("/")[-1])),
}

object_generators = {}

def generate_from_property_key(k, prop,prop_name):
    print("--" + k)
    if k in property_generators:
        return property_generators[k]((prop,prop_name))
    elif prop[k] in property_generators:
        return property_generators[prop[k]]((prop,prop_name))
    
           
def generate_property_value(prop, prop_name):    
    if prop_name in property_generators:
        val = property_generators[prop_name]((prop,prop_name))
        if val is not None:
            print("---" + str(val))
            return val
    
    # iterate through object keys for the property
    for key in prop.keys():
        val = generate_from_property_key(key, prop, prop_name)
        if val is not None:
            print("---" + str(val))
            return val
        else:
            print("---NONE")
    raise Exception(prop)

def get_generator_for_definition(definition_name):
    definition = swagger['definitions'][definition_name] 
    def object_generator():
        instance = {}
        
        # iterate through properties 
        for prop_name in definition['properties']:
            print("-" + prop_name)
            prop = definition['properties'][prop_name]
            try:
                generated_prop = generate_property_value(prop, prop_name)
            except Exception as e:
                print(prop_name)
                raise e
            instance[prop_name] = generated_prop
        yield instance
    return object_generator
    
for definition_name in swagger["definitions"]:
    object_generators[definition_name] = get_generator_for_definition(definition_name)

def generate(obj_name):
    if obj_name not in object_generators:
        raise Exception("Object " + obj_name + " is unknown")
    return object_generators[obj_name]()

In [40]:
list(generate("Order"))

-id
---82903d7f-3b24-4860-9bf3-212028c08173
-petId
--type
---100
-quantity
---67
-shipDate
--type
---NONE
--format
---2017-04-21 14:37:09
-status
--type
---NONE
--description
---NONE
--enum
---approved
-complete
--type
---True


[{'id': '82903d7f-3b24-4860-9bf3-212028c08173',
  'petId': 100,
  'quantity': 67,
  'shipDate': datetime.datetime(2017, 4, 21, 14, 37, 9),
  'status': 'approved',
  'complete': True}]